In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Crew, Agent, Task

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
data_analyst_agent = Agent(
    role="Senior Data Analyst",
    goal="Monitor & Analyze market data in real-time to identify trends & predict market movements.",
    backstory=(
    "Specializing in financial markets, this agent uses statistical modeling & machine learning to provide crucial insights."
    "With a knack for data, the Data Analyst Agent is the cornerstone for informing trading decisions."
  ),
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
trading_strategy_agent = Agent(
  role="Trading Strategy Developer",
  goal="Develop & test various trading strategies based on insights from the Data Analyst Agent.",
  backstory=(
    "Equipped with a deep understanding of financial markets & quantitative analysis, this Agent devices & refines trading strategies."
    "It evaluate the performance of different approaches to determine the  most profitable & risk-averse options."
  ),
  verbose=True,
  allow_delegation=True,
  tools = [scrape_tool, search_tool]
)

In [10]:
execution_agent=Agent(
    role="Trade Advisor",
  goal="Suggest optimal trade execution strategies based on approved trading strategies.",
  backstory=(
    "This Agent specializes in Analyzing the timing, price & logistical details of potential trades."
    "By Evaluating these factors, it provides well-founded suggestions for when & how trades should be executed to maximize efficiency & adherence to strategy."
  ),
  verbose=True,
  allow_delegation=True,
  tools = [scrape_tool, search_tool]
)

In [11]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating TAsks

In [12]:
# Task for Data Analysis Agent
data_analysis_task = Task(
    description=(
        "Continously monitor & Analyze market data for the selected stock ({stock_selection})."
        "Use statistical modeling & machine learning to identify trends & predict market movements."
    ),
    expected_output=(
        "Insights & alerts about significant market"
        "opportunities or threats for {stock_selection}"
    ),
    agent=data_analyst_agent
)

In [13]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develope & refine trading strategies based on the insights from the"
        "Data Analyst & user-defined risk tolerance ({risk_tolerance})."
        "Consider trading preferences ({trading_strategy_preference})"
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [14]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [15]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [16]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4o-mini", 
                           temperature=0.5),
    process=Process.hierarchical,
    verbose=True
)

In [17]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [18]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continously monitor & Analyze market data for the selected stock (AAPL).Use statistical modeling & machine learning to identify trends & predict market movements.
 

I encountered an error while trying to use the tool. This was the error: 'SERPER_API_KEY'.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Crew Manager
## Thought: To begin monitoring and analyzing market data for AAPL, I need to gather relevant market data and potentially identify resources that provide insights related to stock analysis.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"AAPL stock market analysis trends

In [19]:
from IPython.display import Markdown

Markdown(result.raw)

**Risk Analysis Report for AAPL Trading Strategies**

1. **Momentum Trading Risks:**
   - **Market Volatility:** Momentum traders capitalize on stock price trends, but increased volatility can lead to unexpected price reversals.
   - **Mitigation:** Implement strict stop-loss orders to limit potential losses and monitor momentum indicators closely to adjust positions swiftly.

2. **Scalping Risks:**
   - **Liquidity Issues:** Scalping involves rapid buy/sell actions, which can be impeded by liquidity gaps, particularly in volatile markets.
   - **Mitigation:** Focus on high-volume trading times and highly liquid stocks. Use limit orders to ensure execution at desired price points.

3. **Breakout Trading Risks:**
   - **False Breakouts:** Traders might enter on a breakout only to face a reversal, leading to losses.
   - **Mitigation:** Validate breakouts by waiting for confirmation through volume analysis and employ trailing stop-loss orders to protect gains.

4. **Reversal Trading Risks:**
   - **Sudden Price Movements:** Attempting to predict reversals can lead to significant losses if the trend continues unexpectedly.
   - **Mitigation:** Use reversal patterns in conjunction with oscillators like RSI for confirmation. Set tight stop-loss limits to exit quickly if the trade fails.

5. **Technical Indicators Analysis Risks:**
   - **Over-reliance on Indicators:** Indicators can lag and may not account for sudden market news affecting prices.
   - **Mitigation:** Combine multiple indicators and complement them with fundamental analysis for a well-rounded approach. Regularly review strategy effectiveness.

6. **News-Based Trading Risks:**
   - **Impact from News:** Rapid news developments can lead to swift and volatile price changes impacting positions adversely.
   - **Mitigation:** Stay updated with real-time news feeds. Use options like puts or calls to hedge positions and set stop-loss orders aligned with expected volatility.

7. **Daily Chart Patterns Execution Risks:**
   - **Pattern Failures:** Relying on chart patterns alone might result in misinterpretation due to atypical price behavior.
   - **Mitigation:** Confirm patterns with technical indicators and adopt a flexible strategy that allows for quick adjustment to market changes.

**Conclusion:**
Successful trading requires a dynamic approach that incorporates effective risk management techniques. Implementing stop-loss orders, diversifying trades, adjusting strategies based on real-time data, and staying informed on market conditions are crucial to aligning trading activities with risk tolerance.
```